In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [99]:
%load_ext autoreload
import sys
if not '/Users/nlarusstone/Documents/MPhil/Research/cf_fba' in sys.path:
    sys.path.append('/Users/nlarusstone/Documents/MPhil/Research/cf_fba')
import src.utils as utils
%autoreload 2

In [2]:
df = pd.read_csv('../data/17_5_18_T7_mRFP_NLS.CSV', skiprows=6)
print df.shape
gain_diff = df.shape[0] / 5
times = df["Unnamed: 1"]
df.drop('Unnamed: 1', inplace=True, axis=1)
# Bad data
df.drop(['E09', 'F04', 'F05'], inplace=True, axis=1)
df.head()

(1591, 38)


,Well,B02,B03,B04,B05,B06,B07,B08,B09,C02,...,D09,E02,E03,E04,E05,E06,E07,E08,F02,F03
0,Raw Data (570-20/630-40 4),586.0,266.0,620.0,628.0,725.0,616.0,735.0,570.0,637.0,...,633.0,509.0,614.0,354.0,538.0,720.0,291.0,522.0,421.0,531.0
1,Raw Data (570-20/630-40 4),633.0,245.0,625.0,653.0,749.0,614.0,717.0,582.0,647.0,...,630.0,529.0,631.0,342.0,552.0,714.0,310.0,491.0,418.0,539.0
2,Raw Data (570-20/630-40 4),614.0,268.0,646.0,629.0,742.0,616.0,737.0,559.0,642.0,...,625.0,550.0,626.0,354.0,570.0,712.0,313.0,531.0,431.0,532.0
3,Raw Data (570-20/630-40 4),647.0,310.0,701.0,642.0,801.0,698.0,815.0,606.0,640.0,...,642.0,513.0,628.0,343.0,610.0,700.0,362.0,535.0,436.0,503.0
4,Raw Data (570-20/630-40 4),611.0,294.0,689.0,683.0,794.0,677.0,798.0,663.0,667.0,...,650.0,493.0,615.0,316.0,583.0,677.0,334.0,533.0,427.0,539.0


In [5]:
gain2 = gain_diff
outs = df[gain2:gain2+gain_diff - 1].mean(axis=0)
#outs_norm = outs / outs.max()
#print outs_norm
#sor_args = outs_norm.argsort()
#outs_norm[sor_args]

In [71]:
df = pd.read_csv('../data/17_5_18_T7_mRFP_NLS.CSV', skiprows=6)
print df.shape
gain_diff = df.shape[0] / 5
times = df["Unnamed: 1"]
df.drop('Unnamed: 1', inplace=True, axis=1)
# Bad data
df.drop(['E09', 'F04', 'F05'], inplace=True, axis=1)
df.head()
gain2 = gain_diff
outs = df[gain2:gain2+gain_diff - 1].mean(axis=0)
conds = pd.read_csv('../data/17_5_18_exp_conditions.csv')
conds.drop(conds.shape[0] - 1, inplace=True)
conds_full = conds.reindex(np.repeat(conds.index.values, 2)).reset_index()
conds_full = conds_full.drop(32).reset_index(drop=True)
conds_full['OUT'] = outs.reset_index(drop=True)
conds_avg = conds_full.groupby('index').mean()
conds_norm = conds_avg / conds_avg.max()
conds_norm.to_csv('../data/17_5_18_EXPERIMENT.csv')
conds_norm

,mdx,pi,nts,k,OUT
index,,,,,
0,0.00,0.00,1.00,0.00,0.566331
1,0.00,1.00,0.00,0.00,0.523761
2,1.00,0.00,0.00,0.00,0.589774
3,0.00,0.50,0.50,0.00,0.969988
4,0.50,0.00,0.50,0.00,0.758090
5,0.50,0.50,0.00,0.00,0.197350
6,0.25,0.25,0.50,0.00,0.404324
7,0.25,0.50,0.25,0.00,1.000000
8,0.50,0.25,0.25,0.00,0.958974


In [18]:
# amt in g, vol in mL, mw in g/mol
def calc_conc(amt, vol, mw=None, seq=None, seq_type=None):
    # seq can be DNA or protein or an amino acid
    if seq:
        mw = su.molecular_weight(seq, seq_type)
    elif not mw:
        raise Exception('Need a molecular weight for non-DNA')
    conc = (amt * 1000) / (vol * mw)
    # returns Molar concentrations
    return conc

def conc_dilution(start_conc, vol_add, tot_vol):
    return start_conc * (vol_add / tot_vol)

In [56]:
def read_start_conds():
    aa_mix = pd.read_csv('../data/aa_mix.csv', index_col='AA')
    nrg_mix = pd.read_csv('../data/energy_mix.csv', index_col='compound')
    with open('../genes/rfp.txt', 'r') as f:
        seq = f.read()
    cfps_conc = pd.read_csv('../data/cfps_start.csv', index_col='compound')
    dna_conc = calc_conc(0.000750, 0.00496, seq=seq, seq_type='DNA')
    return aa_mix, nrg_mix, cfps_conc, dna_conc
aa_mix, nrg_mix, cfps_conc, dna_conc = read_start_conds()
rxn_amt = 5
batch_size = 50 / rxn_amt
cfps_conc['amt'] = cfps_conc['amt'] / batch_size

In [57]:
aa_mix['start_conc'] = aa_mix.apply(lambda row: calc_conc(row['weight_add'], 1, 
                                                          seq=pdb.three_to_one(row.name.upper()), seq_type='protein'), axis=1)
aa_mix['conc_add'] = conc_dilution(aa_mix['start_conc'], aa_mix['vol_add'], aa_mix['vol_add'].sum())

In [58]:
pi_conc = calc_conc(0.15, 5, mw=611.77)

In [60]:
import Bio.PDB.Polypeptide as pdb
import Bio.SeqUtils as su

nrg_mix['start_conc'] = nrg_mix.apply(lambda row: calc_conc(row['amt'], row['fill'], mw=row['mw']), axis=1)
nrg_mix['conc_add'] = conc_dilution(nrg_mix['start_conc'], nrg_mix['vol_add'], nrg_mix['vol_add'].sum())

for cmpnd, vals in nrg_mix.iterrows():
    cfps_conc.loc[cmpnd] = [vals['conc_add'], (5.0 / batch_size)]
for aa, vals in aa_mix.iterrows():
    cfps_conc.loc[aa] = [vals['conc_add'], (10.0 / batch_size)]
cfps_conc.loc['GENE'] = [dna_conc, (4.96 / batch_size)]
#cfps_conc['final_conc'] = conc_dilution(cfps_conc['start_conc'], cfps_conc['amt'], rxn_amt)

In [72]:
for idx, row in conds_norm.iterrows():
    cfps_conc_tmp = cfps_conc.copy()
    cfps_conc_tmp.loc['pi']['amt'] += row['pi']
    cfps_conc_tmp.loc['k']['amt'] += row['k']
    for nt in ['atp', 'gtp', 'ctp', 'utp']:
        cfps_conc_tmp.loc[nt]['amt'] += row['nts'] / 4
    for cmpnd, vals in nrg_mix.iterrows():
        cfps_conc_tmp.loc[cmpnd] = [vals['conc_add'], (5.0 + row['mdx'] / batch_size)]
    rxn_amt += 1
    exp_concs = conc_dilution(cfps_conc_tmp['start_conc'], cfps_conc_tmp['amt'], rxn_amt)

In [102]:
def change_conc(model, cfps_conc):
    mod = model.copy()

    for metab, vals in cfps_conc.iteritems():
        flux = utils.conc_to_flux(vals) * 100

        if metab == 'trna':
            ms = model.metabolites.query('trna')
        elif metab.upper() in pdb.aa3:
            ms = get_aa_metab(model, metab.lower(), cmpt='c')
        else:
            ms = mod.metabolites.query(r'^{0}_c'.format(metab))
        for m in ms:
            rxn_nm = 'EX_' + m.id
            rxn = mod.reactions.get_by_id(rxn_nm)
            rxn.upper_bound = flux
            #mod.add_boundary(metabolite=m, type='exchange', lb=0, ub=flux)
            #mod.add_boundary(metabolite=m, type='cfps-medium', reaction_id=rxn_nm, lb=0, ub=flux) 
    return mod


In [103]:
change_conc(model, cfps_conc)

Name,iJO1366
Memory address,0x0120cd04d0
Number of metabolites,1805
Number of reactions,2583
Objective expression,1.0*BIOMASS_Ec_iJO1366_core_53p95M - 1.0*BIOMASS_Ec_iJO1366_core_53p95M_reverse_5c8b1
Compartments,"periplasm, cytosol, extracellular space"


In [95]:
model = cobra.io.read_sbml_model('../models/iJO1366.xml')